In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [ ]:
sr = 22050 # isti za sve pjesme

data = pd.DataFrame(index=np.array([int(filename[:-4]) for dirname, _, filenames in os.walk('/kaggle/input/fma-small-s-db') for filename in filenames]).astype(np.int32))

def load(song_id):
    return np.load(f'/kaggle/input/fma-small-s-db/S_DB/{song_id}.npy')

data['width'] = [load(track_id).shape[1] for track_id in data.index]

In [ ]:
from sklearn.model_selection import train_test_split

height = 128
width = 16
overlap = 0

# Izbacimo pjesme duljine manje od width
data = data[data['width'] >= width]

# Učitajmo podatke o žanrovima
data['genre_name'] = pd.read_csv('/kaggle/input/fma-small/fma_metadata/fma_metadata/tracks.csv', index_col=0, header=[0, 1]).loc[data.index, ('track', 'genre_top')]
genre_encoder = LabelEncoder()
data['genre'] = genre_encoder.fit_transform(data['genre_name']).astype(np.int8)

train_ids, test_ids = train_test_split(data.index, test_size=0.2, stratify=data['genre'], random_state=0)
train_ids, val_ids = train_test_split(train_ids, test_size=0.1, stratify=data.loc[train_ids, 'genre'], random_state=0)

def load_segments(track_ids):
    n_segments = sum(1 + (track_width - width) // (width - overlap) for track_width in data.loc[track_ids, 'width'])
    x = np.empty((n_segments, height, width), dtype=np.float32) # isječci
    y = np.empty((n_segments,), dtype=np.int8)                  # žarn isječka
    z = np.empty((n_segments,), dtype=np.int32)                 # iz koje pjesme isjecak dolazi
    j = 0
    for track_id in track_ids:
        track = load(track_id)
        for end in range(width, track.shape[1], width-overlap):
            x[j] = track[:, end-width:end]
            y[j] = data.loc[track_id, 'genre']
            z[j] = track_id
            j += 1
    x = x.reshape((n_segments, height, width, 1))
    return x, y, z

x_train, y_train, _ = load_segments(train_ids)
x_val, y_val, _ = load_segments(val_ids)
x_test, y_test, z_test = load_segments(test_ids)

In [ ]:
minimum = x_train.min()
maximum = x_train.max()
I=maximum-minimum

In [ ]:
x_train=(x_train-minimum)/I
x_test = (x_test-minimum)/I
x_val = (x_val-minimum)/I

In [ ]:
import tensorflow as tf

tf.compat.v1.disable_eager_execution()

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, InputLayer, UpSampling2D
from tensorflow.keras.callbacks import TensorBoard
import gc


#convlayers=1 (64)
#convlayers=1 (128)
#convlayers=1 (64) denselayers=1 (16)
#convlayers=1 (64) denselayers=1 (32)
#convlayers=1 (128) denselayers=1 (16)
#convlayers=1 (128) denselayers=1 (32)
#convlayers=2 (32) (64)
#convlayers=2 (32) (64) denselayers=1 (16)
#convlayers=2 (32) (64) denselayers=2 (8) (16)
#convlayers=3 (16) (32) (64)
#convlayers=3 (32) (64) (128) denselayers=1 (16)


modeli = [
    
    Sequential([
        InputLayer(input_shape=(height, width, 1)),
        Conv2D(64, (5, 5), activation='relu', padding='same', strides=1),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Flatten(),
        Dense(8, activation="softmax")
    ]),
    
        
    Sequential([
        InputLayer(input_shape=(height, width, 1)),
        Conv2D(128, (5, 5), activation='relu', padding='same', strides=1),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Flatten(),
        Dense(8, activation="softmax")
    ]),
    
    Sequential([
        InputLayer(input_shape=(height, width, 1)),
        Conv2D(64, (5, 5), activation='relu', padding='same', strides=1),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Flatten(),
        Dense(16, activation="relu"),
        Dense(8, activation="softmax")
    ]),
    
    Sequential([
        InputLayer(input_shape=(height, width, 1)),
        Conv2D(64, (5, 5), activation='relu', padding='same', strides=1),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Flatten(),
        Dense(32, activation="relu"),
        Dense(8, activation="softmax")
    ]),
    
    Sequential([
        InputLayer(input_shape=(height, width, 1)),
        Conv2D(128, (5, 5), activation='relu', padding='same', strides=1),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Flatten(),
        Dense(16, activation="relu"),
        Dense(8, activation="softmax")
    ]),
    
        
    Sequential([
        InputLayer(input_shape=(height, width, 1)),
        Conv2D(128, (5, 5), activation='relu', padding='same', strides=1),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Flatten(),
        Dense(32, activation="relu"),
        Dense(8, activation="softmax")
    ]),
    
        
    Sequential([
        InputLayer(input_shape=(height, width, 1)),
        Conv2D(32, (5, 5), activation='relu', padding='same', strides=1),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Conv2D(64, (5, 5), activation='relu', padding='same', strides=1),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Flatten(),
        Dense(8, activation="softmax")
    ]),
    
    Sequential([
        InputLayer(input_shape=(height, width, 1)),
        Conv2D(32, (5, 5), activation='relu', padding='same', strides=1),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Conv2D(64, (5, 5), activation='relu', padding='same', strides=1),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Flatten(),
        Dense(16, activation="relu"),
        Dense(8, activation="softmax")
    ]),
    
    Sequential([
        InputLayer(input_shape=(height, width, 1)),
        Conv2D(32, (5, 5), activation='relu', padding='same', strides=1),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Conv2D(64, (5, 5), activation='relu', padding='same', strides=1),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Flatten(),
        Dense(8, activation="relu"),
        Dense(16, activation="relu"),
        Dense(8, activation="softmax")
    ]),
    
    Sequential([
        InputLayer(input_shape=(height, width, 1)),
        Conv2D(16, (5, 5), activation='relu', padding='same', strides=1),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Conv2D(32, (5, 5), activation='relu', padding='same', strides=1),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Conv2D(64, (5, 5), activation='relu', padding='same', strides=1),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Flatten(),
        Dense(8, activation="softmax")
    ]),
    
    Sequential([
        InputLayer(input_shape=(height, width, 1)),
        Conv2D(32, (5, 5), activation='relu', padding='same', strides=1),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Conv2D(64, (5, 5), activation='relu', padding='same', strides=1),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Conv2D(128, (5, 5), activation='relu', padding='same', strides=1),
        MaxPooling2D(pool_size=(2, 2), strides=2),
        Flatten(),
        Dense(16, activation="relu"),
        Dense(8, activation="softmax")
    ])
    
    
]

In [ ]:
checkpoints = [tf.keras.callbacks.ModelCheckpoint(
    filepath=f'/kaggle/working/checkpoint{k}',
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True) for k in range(1,12)]   
    

In [ ]:
from keras import backend as K

sgd = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)


for i in range(11):
    print(f'Treniranje modela broj {i+1}')
    modeli[i].compile(optimizer = sgd,
    loss = 'sparse_categorical_crossentropy',
    metrics=['accuracy'])
    modeli[i].fit(x_train, y_train, validation_data=(x_val, y_val), epochs=70, batch_size=256, callbacks=[checkpoints[i]])
    modeli[i].load_weights(checkpoints[i].filepath)
    modeli[i].save(f'model{i+1}.h5')
    gc.collect()
    print("\n")

In [ ]:
for i in range(11):
    print(modeli[i].evaluate(x_val,y_val)[1])

In [ ]:
best_model_number = 0
max_val_accuracy = modeli[0].evaluate(x_val,y_val)[1]

for i in range(1,11):
    temp_val_accuracy = modeli[i].evaluate(x_val,y_val)[1]
    
    if max_val_accuracy < temp_val_accuracy:
        best_model_number = i
        max_val_accuracy = temp_val_accuracy


In [ ]:
from sklearn.metrics import accuracy_score


pred = pd.DataFrame(modeli[best_model_number].predict(x_test), index=z_test)
# Svaki žanr glasa za isječak s maksimalnom vjerojatnosti, zatim gledamo za koji žanr najviše isječaka glasa
votes = pred.idxmax(axis=1)
score=accuracy_score(data.loc[test_ids, 'genre'].sort_index(), votes.groupby(votes.index).agg(lambda x:x.value_counts().index[0]))

In [ ]:
print(f"točnost na test skupu je: {score}")